# Arcane Python syntax

Python has a syntax with a few features that are quite unique.

**General advice:** don't use any of this unless you feel comfortable with it, since mistakes can lead to bugs that are hard to track down.

In [1]:
from traceback import print_exc

## Interval checking

In Python an expression such as `a < x <= b` is legal and well-defined.

In [2]:
a, b = 1, 3
for x in range(-2, 5):
    if a < x <= b:
        print(f'{a} < {x} <= {b}: True')
    else:
        print(f'{a} < {x} <= {b}: False')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 < 0 <= 3: False
1 < 1 <= 3: False
1 < 2 <= 3: True
1 < 3 <= 3: True
1 < 4 <= 3: False


The code above can be simplified to:

In [3]:
a, b = 1, 3
for x in range(-2, 5):
    print(f'{a} < {x:2d} <= {b}: {a < x <= b}')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 <  0 <= 3: False
1 <  1 <= 3: False
1 <  2 <= 3: True
1 <  3 <= 3: True
1 <  4 <= 3: False


## Multiple equality, inequality

Along the same lines, `a == b == x` is also legal and well-defined.

In [4]:
for a in range(3):
    for b in range(3):
        print(f'{a} == {b} == 1: {a == b == 1}')

0 == 0 == 1: False
0 == 1 == 1: False
0 == 2 == 1: False
1 == 0 == 1: False
1 == 1 == 1: True
1 == 2 == 1: False
2 == 0 == 1: False
2 == 1 == 1: False
2 == 2 == 1: False


Although `a != b != x` is legal as well, it may, at least at first sight, not behave as expected.

In [5]:
for a in range(3):
    for b in range(3):
        print(f'{a} != {b} != 1: {a != b != 1}')

0 != 0 != 1: False
0 != 1 != 1: False
0 != 2 != 1: True
1 != 0 != 1: True
1 != 1 != 1: False
1 != 2 != 1: True
2 != 0 != 1: True
2 != 1 != 1: False
2 != 2 != 1: False


From the above, it is clear that `a != b != x` translates to `a != b and b != c`, which is true when `a == c and a != b`.  From a mathematical point of view, bear in mind that `==` is transitive, while `!=` is not.

## Iteration with `else`

Iteration statements in Python, i.e., `for` and `while` can have an `else` block.  The latter is executed when the iteration statement terminates normally, i.e., not by a `break` statement.

In [6]:
def illustrate_for_else(x):
    for i in range(10):
        if i == x:
            print('break')
            break
    else:
        print('no break')

In [7]:
illustrate_for_else(12)

no break


In [8]:
illustrate_for_else(5)

break


Although naming this syntactic construct `else` feels awkward, it is quite useful, since it is a syntactic shortcut for the following reasonably common construct.

In [9]:
def illustrate_for_bool(x):
    completed_succesfully = True
    for i in range(10):
        if i == x:
            print('break')
            completed_succesfully = False
            break
    if completed_succesfully:
        print('no break')

In [10]:
illustrate_for_bool(12)

no break


In [11]:
illustrate_for_bool(5)

break


The execution of `continue` has no influence on this.

In [12]:
for i in range(5):
    if i > -1:
        continue
    print(f'did something for {i}')
else:
    print('completed normally')

completed normally


The `while` statement can have an `else` with the same semantics.

## Logical shortcircuits

Boolean operators can be used directly as control statements.  This is familiar to experience Bash programmers, but leads to code that is somewhat hard to understand.

In [13]:
import sys

In [14]:
output_file = None
fh = output_file or sys.stdout
print('hello', file=fh)

hello


If the first operand to `or` is `False`, the value of the expression will be that of the second operand. The value `None` converts to Boolean `False`, hence the behavior above. However, if the first operand can be converted to `True`, the value of the expression is that of the first operand, a file handle in the example below.

In [15]:
output_file = open('remove_me.txt', 'w')
fh = output_file or sys.stdout
print('hello', file=fh)

In [16]:
!cat remove_me.txt

The semantics of `and` expressions is similar.  If the first operand converts to `True`, the expression will have the value of the second operand. If the first operand converts to `False`, that operand will be the value of the expression.

In [17]:
a_list = []
b_list = [3, 5, 7]
my_list = a_list and b_list
print(my_list)

[]


In [18]:
a_list = [3, 5, 7]
b_list = [3, 5, 7, 9]
my_list = a_list and b_list
print(my_list)

[3, 5, 7, 9]


## Tuple unpacking

Python support tuple unpacking, a feature that can simplify code considerably. Consider a tuble that represents a person as her last name, first name and age.

In [19]:
alice = ('Armstrong', 'Alice', 35)

Now define a function that takes a person as an argument, and prints a greeting.

In [20]:
def greet(person):
    last_name, first_name, _ = person
    print(f'Hello {first_name} {last_name}')

In [21]:
greet(alice)

Hello Alice Armstrong


In the assignment to `last_name` and `first_name`, the first and second field of the tuple are assigned to those variables respectively.  The third field is assigned to `_`, which indicates that we don't care about subsequent fields.

This would break if we would decide to enricht tuples representing poople with an extra field, e.g., their passport ID.

In [22]:
alice = ('Armstrong', 'Alice', 35, 548598594)

In [23]:
try:
    greet(alice)
except:
    print_exc()

Traceback (most recent call last):
  File "<ipython-input-23-c8f570cf4b94>", line 2, in <module>
    greet(alice)
  File "<ipython-input-20-9dc596b498dc>", line 2, in greet
    last_name, first_name, _ = person
ValueError: too many values to unpack (expected 3)


Note that there must be as many variables on the left hand side of the assignment operator as values to unpack on the right hand side.  This may in some circumstances be mitigated by using the `*_` placeholder that will match any number of values, i.e.,

In [24]:
last_name, first_name, *_ = alice
f'{first_name} {last_name}'

'Alice Armstrong'

The `*_` placeholder can also be used in the middle, e.g.,

In [25]:
last_name, *_, identification = alice
f'{identification}: {last_name}'

'548598594: Armstrong'

Note that this actually works for other data types besides tuples, for instance for `str`.

In [26]:
s = 'Hello world!'
first_char, *_, last_char, _ = s
first_char, last_char

('H', 'd')

In the assingment above, the string `s` is unpacked into its characters.  The first is assigned to `first_char`, the last (the '!') is ignored due to the `_`, hthe one before last is assigned to `last_char`, and all other characters are ignored due to `*_`.

In [27]:
l = [3, 5, 7, 9]
*_, last = l
last

9

## Unpacking argument lists

Consider a function that takes four arguments `x`, `a`, `b`, and `c`, and computes the polynomial `a*x**2 + b*x + c`.

In [28]:
def f(x, a, b, c):
    return a*x**2 + b*x + c

The function can be called in the usual way, e.g.,

In [29]:
f(3.0, 1.0, -2.0, 0.5)

3.5

However, suppose that the coefficients are stored in a list, i.e.,

In [30]:
coeffs = [1.0, -2.0, 0.5]

The function can conveniently be called by unpacking the list to arguments using the `*` operator.

In [31]:
f(3.0, *coeffs)

3.5

Imagine that the coefficients are stored in a dictionary rather than a list.

In [32]:
coeffs = {'a': 1.0, 'b': -2.0, 'c': 0.5}

Again, argument list expansion can be used, but now with the `**` operator to expand a dictionary.

In [33]:
f(3.0, **coeffs)

3.5

This can be exploited if the function is default arguments, e.g.,

In [34]:
def f(x, a=0.0, b=1.0, c=0.0):
    return a*x**2 + b*x + c

We can now call the function using argument list expansion on a dictrionary that contains only those coefficients that should not have the function's default values.

In [35]:
coeffs = {'a': -2.0, 'c': 7.0}

In [36]:
f(3.0, **coeffs)

-8.0

## Functions with variable number of arguments

Consider a function that copmutes the minimum value of its arguments, at least two.

In [37]:
def minimum(a, *args):
    if args:
        b = min(args)
        return a if a < b else b
    else:
        return a

This function can be called with an arbirtrary number of arguments, but at least one.

In [38]:
minimum(5)

5

In [39]:
minimum(3, 1)

1

In [40]:
minimum(4, 9, -1, 7, 15, 3)

-1

In [41]:
try:
    minimum()
except:
    print_exc()

Traceback (most recent call last):
  File "<ipython-input-41-2c1f2e1b9987>", line 2, in <module>
    minimum()
TypeError: minimum() missing 1 required positional argument: 'a'


The `*` indicates that a sequence of zero or more arguments can be given to the function, and they will be stored in the tuple `args`.

In [42]:
from datetime import datetime
import sys

Similarly, we can use `**` to indicate that the function takes an arbitrary number of keyword arguments.

In [43]:
def print_log_entry(file, **kwargs):
    print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}:', end='', file=file)
    for k, v in kwargs.items():
        print(f' {k}="{v}"', end='', file=file)

In [44]:
print_log_entry(sys.stdout, a=25, b='abc')

2019-12-08 08:07:24: a="25" b="abc"

You can combine the various type of variables, but they have to be specified in the order
  1. positional arguments,
  2. `*args`,
  3. `**kwargs`.